In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
zip_file = 'drive/MyDrive/AI/CNN/numbers/Numbers_Images_Dataset.zip'
with zipfile.ZipFile(zip_file,'r') as zip_ref:
    zip_ref.extractall('images')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
directory_path = "/content/images"
images_df = pd.read_csv(directory_path + "/numbers.csv")
images_df

origin    group  label                                           file
0      chars74k  GoodImg      8  chars74k_png/GoodImg/Sample8/img009-00029.png
1      chars74k  GoodImg      8  chars74k_png/GoodImg/Sample8/img009-00032.png
2      chars74k  GoodImg      8  chars74k_png/GoodImg/Sample8/img009-00020.png
3      chars74k  GoodImg      8  chars74k_png/GoodImg/Sample8/img009-00027.png
4      chars74k  GoodImg      8  chars74k_png/GoodImg/Sample8/img009-00022.png
...         ...      ...    ...                                            ...
71585     mnist      Hnd      0                mnist_png/Hnd/Sample0/45577.png
71586     mnist      Hnd      0                mnist_png/Hnd/Sample0/19491.png
71587     mnist      Hnd      0                mnist_png/Hnd/Sample0/30425.png
71588     mnist      Hnd      0                mnist_png/Hnd/Sample0/11240.png
71589     mnist      Hnd      0                mnist_png/Hnd/Sample0/49278.png

[71590 rows x 4 columns]

In [ ]:
from pathlib import Path

def create_directory(directory_path):
    # Create a directory if it doesn't exist
    try:
        Path(directory_path).mkdir(parents=True, exist_ok=True)
        print(f"Directory '{directory_path}' created successfully.")
    except OSError as e:
        print(f"Error creating directory: {e}")

import shutil

def copy_file(source_path, destination_path):
    try:
        shutil.copy(source_path, destination_path)
        print(f"File copied from '{source_path}' to '{destination_path}' successfully.")
    except Exception as e:
        print(f"Error copying file: {e}")

In [ ]:
create_directory(directory_path + "/" + "test")
create_directory(directory_path + "/" + "train")
folders = []
for i in range(0,10):
    folders.append(f"Sample{i}")
    create_directory(f'{directory_path}/train/Sample{i}')
    create_directory(f'{directory_path}/test/Sample{i}')

Directory '/content/images/test' created successfully.
Directory '/content/images/train' created successfully.
Directory '/content/images/train/Sample0' created successfully.
Directory '/content/images/test/Sample0' created successfully.
Directory '/content/images/train/Sample1' created successfully.
Directory '/content/images/test/Sample1' created successfully.
Directory '/content/images/train/Sample2' created successfully.
Directory '/content/images/test/Sample2' created successfully.
Directory '/content/images/train/Sample3' created successfully.
Directory '/content/images/test/Sample3' created successfully.
Directory '/content/images/train/Sample4' created successfully.
Directory '/content/images/test/Sample4' created successfully.
Directory '/content/images/train/Sample5' created successfully.
Directory '/content/images/test/Sample5' created successfully.
Directory '/content/images/train/Sample6' created successfully.
Directory '/content/images/test/Sample6' created successfully.


In [ ]:
for folder in folders:
    images_df['file'] = images_df['file'].astype('string')
    df = images_df[images_df['file'].str.contains(folder,case=False,na=False)]
    ten_percent = int(df.shape[0]/10)
    print(ten_percent)
    for i in range(0,ten_percent):
        source = directory_path + "/numbers/" + df['file'].iloc[i]
        dest = directory_path + "/"  + "test" + "/" + folder
        copy_file(source,dest)

    for i in range(ten_percent,len(df)):
        source = directory_path + "/numbers/" + df['file'].iloc[i]
        dest = directory_path + "/"  + "train" + "/" + folder
        copy_file(source,dest)

Streaming output truncated to the last 5000 lines.
File copied from '/content/images/numbers/mnist_png/Hnd/Sample9/48996.png' to '/content/images/train/Sample9' successfully.
File copied from '/content/images/numbers/mnist_png/Hnd/Sample9/12131.png' to '/content/images/train/Sample9' successfully.
File copied from '/content/images/numbers/mnist_png/Hnd/Sample9/57236.png' to '/content/images/train/Sample9' successfully.
File copied from '/content/images/numbers/mnist_png/Hnd/Sample9/17226.png' to '/content/images/train/Sample9' successfully.
File copied from '/content/images/numbers/mnist_png/Hnd/Sample9/50833.png' to '/content/images/train/Sample9' successfully.
File copied from '/content/images/numbers/mnist_png/Hnd/Sample9/34227.png' to '/content/images/train/Sample9' successfully.
File copied from '/content/images/numbers/mnist_png/Hnd/Sample9/44429.png' to '/content/images/train/Sample9' successfully.
File copied from '/content/images/numbers/mnist_png/Hnd/Sample9/8322.png' to '/co

In [ ]:
tf.config.set_visible_devices([],'GPU')
print('Available GPU cores ', len(tf.config.experimental.list_physical_devices('GPU')))

Available GPU cores  1


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data_genarator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    )

test_data_generator = ImageDataGenerator(
    rescale=1./255
    )

In [ ]:
train_data = train_data_genarator.flow_from_directory(
    directory_path + "/" + 'train',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)

test_data = test_data_generator.flow_from_directory(
    directory_path + "/" + 'test',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)

Found 64344 images belonging to 10 classes.
Found 6771 images belonging to 10 classes.


In [ ]:
cnn = tf.keras.models.Sequential()
# First convolutional layer

cnn.add(tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation='relu',
    input_shape=[64,64,3]
    )
)

cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='valid'
    )
)

# hidden convolutional layer

cnn.add(tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation='relu'
    )
)

cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='valid'
    )
)


cnn.add(tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation='relu'
    )
)

cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='valid'
    )
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
cnn.add(tf.keras.layers.Flatten())

In [ ]:
cnn.add(tf.keras.layers.Dense(128,activation='relu'))
cnn.add(tf.keras.layers.Dense(128,activation='relu'))
# Output layer
cnn.add(tf.keras.layers.Dense(10,activation='relu'))

In [ ]:
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
cnn.fit(train_data,validation_data=test_data,epochs=30,batch_size=32)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2011/2011 ━━━━━━━━━━━━━━━━━━━━ 631s 312ms/step - accuracy: 0.4775 - loss: 2.6904 - val_accuracy: 0.0922 - val_loss: 2.6918
Epoch 2/30
2011/2011 ━━━━━━━━━━━━━━━━━━━━ 611s 304ms/step - accuracy: 0.6185 - loss: 1.3816 - val_accuracy: 0.0956 - val_loss: 2.2812
Epoch 3/30
2011/2011 ━━━━━━━━━━━━━━━━━━━━ 609s 302ms/step - accuracy: 0.6626 - loss: 1.1954 - val_accuracy: 0.1041 - val_loss: 2.3335
Epoch 4/30
2011/2011 ━━━━━━━━━━━━━━━━━━━━ 628s 305ms/step - accuracy: 0.7075 - loss: 1.0363 - val_accuracy: 0.3067 - val_loss: 2.0881
Epoch 5/30
2011/2011 ━━━━━━━━━━━━━━━━━━━━ 629s 312ms/step - accuracy: 0.8216 - loss: 0.7758 - val_accuracy: 0.3101 - val_loss: 2.0215
Epoch 6/30
2011/2011 ━━━━━━━━━━━━━━━━━━━━ 705s 324ms/step - accuracy: 0.8032 - loss: 0.7878 - val_accuracy: 0.6962 - val_loss: 1.2963
Epoch 7/30
2011/2011 ━━━━━━━━━━━━━━━━━━━━ 657s 311ms/step - accuracy: 0.8788 - loss: 0.5777 - val_accuracy: 0.7225 - val_loss: 1.2655
Epoch 8/30
2011/2011 ━━━━━━━━━━━━━━━━━━━━ 670s 305ms/step - accuracy: 0.8

In [ ]:
from joblib import dump
dump(cnn, directory_path + "/model/" + 'numbers.joblib')

In [ ]:
from joblib import load
cnn = load(directory_path + "/model/" + 'numbers.joblib')
from tensorflow.keras.preprocessing import image
directory_path = "drive/MyDrive/AI/CNN/numbers/images/new_data"
images = list_files_in_directory(directory_path)

results = []
for image in images:
    test_img = image.load_img(directory_path + "/new_data/" + images[i])]
    test_image = image.img_to_array(test_img)
    test_image = test_image/255
    test_image = np.expand_dims(test_image,axis=0)
    result = cnn.predict(test_image)
    results.append(result)
print(results)